In [21]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
movies = pd.read_csv('../data/csv/movies.csv')

In [23]:
ratings = pd.read_csv('../data/csv/ratings.csv')

In [24]:
tags = pd.read_csv('../data/csv/tags.csv')

In [25]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [26]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [27]:
tags.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


In [28]:
joined = movies.set_index('movieId').join(ratings.set_index('movieId'))
joined = joined[~joined['userId'].isna()] 
joined

,title,genres,userId,rating,timestamp
movieId,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...
193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
193585,Flint (2017),Drama,184.0,3.5,1.537110e+09


In [29]:
trimmed = joined.reset_index()[['movieId', 'userId', 'rating']]
trimmed

,movieId,userId,rating
0,1,1.0,4.0
1,1,5.0,4.0
2,1,7.0,4.5
3,1,15.0,2.5
4,1,17.0,4.5
...,...,...,...
100831,193581,184.0,4.0
100832,193583,184.0,3.5
100833,193585,184.0,3.5
100834,193587,184.0,3.5


In [30]:
users_set = trimmed['userId'].unique()
users_set.sort()

movie_set = trimmed['movieId'].unique()
movie_set.sort()

user_frame = pd.DataFrame(index = users_set, columns = movie_set)
user_frame = user_frame.fillna(0)

counter = 0
ten_precent = int(trimmed.shape[0] / 10)
for row in trimmed.iterrows():
    counter += 1
    if counter % ten_precent == 0:
        print(counter // ten_precent * 10, '% of the way done')
    row = row[1]
    _userId = row['userId']
    _movieId = row['movieId']
    user_frame.loc[_userId][_movieId] = row['rating']

10 % of the way done
20 % of the way done
30 % of the way done
40 % of the way done
50 % of the way done
60 % of the way done
70 % of the way done
80 % of the way done
90 % of the way done
100 % of the way done


In [31]:
user_frame.head()

,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
1.0,4,0,4,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5.0,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
sims

[(239.0, 0.43167439571758726),
 (580.0, 0.42945345530448664),
 (480.0, 0.38803478357059323),
 (219.0, 0.38404602493858264),
 (434.0, 0.3810636267533063),
 (414.0, 0.3785949743758926),
 (305.0, 0.36575478946461404),
 (590.0, 0.35649334459700144),
 (608.0, 0.3420108458684847),
 (166.0, 0.3402041707585982),
 (254.0, 0.33862565782016046),
 (274.0, 0.3358017896097892),
 (561.0, 0.33575781905060875),
 (18.0, 0.33416287601230354),
 (232.0, 0.32917658966107183),
 (68.0, 0.3291084106813047),
 (140.0, 0.32293694990303434),
 (425.0, 0.32220676938154413),
 (599.0, 0.3160477560352997),
 (573.0, 0.3127277399624082),
 (464.0, 0.31240821489690795),
 (220.0, 0.3117208601366954),
 (282.0, 0.31121451783661386),
 (199.0, 0.3105636581772738),
 (298.0, 0.30937396187745236),
 (354.0, 0.3044227743016517),
 (380.0, 0.30359824193827367),
 (483.0, 0.3032679935129761),
 (249.0, 0.30309080191219684),
 (610.0, 0.3027892502381828),
 (432.0, 0.30261681751495073),
 (330.0, 0.3024611727131426),
 (182.0, 0.3010842480023

In [39]:
user_num = 6
new_user = user_frame.loc[user_num]
new_frame = user_frame.copy().drop(user_num)

sims = []
for row in new_frame.iterrows():
    userId = row[0]
    row = row[1]
    sims.append((userId, cosine_similarity([row], [new_user])[0][0]))
    
sims.sort(key = lambda x: x[1], reverse = True)



def get_top_5(user_frame, user_info, userId_other, movies):
    user_info_other = user_frame.iloc[userId_other]

    m = (user_info_other * user_info)
    top_five = m[m > 0].sort_values(ascending = False)[0: 5]

    for movie_id in top_five.index:
        top_five.loc[movie_id] = movies.set_index('movieId').loc[movie_id]['title']
    return top_five

top_five = []
index = 0
while len(top_five) < 5:
    closest_user = int(sims[index][0])
    new_movies = list(get_top_5(new_frame, new_user, closest_user, movies))
    for movie in new_movies:
        if movie not in top_five:
            top_five.append(movie)
    index += 1
top_five = top_five[:5]

In [40]:
top_five

['Lion King, The (1994)',
 'Forrest Gump (1994)',
 'Shawshank Redemption, The (1994)',
 'Cliffhanger (1993)',
 'Specialist, The (1994)']

In [35]:
#old code
'''
users_set = trimmed['userId'].unique()
users_set.sort()
user_frame = pd.DataFrame(users_set, columns = ['userId'])
user_frame = user_frame.set_index('userId')

movies_set = trimmed['movieId'].unique()
movies_set.sort()
movie_vector_blank = pd.DataFrame([np.zeros(len(movies_set))], columns = movies_set)

user_frame['movie_vect'] = [movie_vector_blank.copy() for i in range(user_frame.shape[0])]

counter = 0
ten_precent = int(trimmed.shape[0] / 10)
for row in trimmed.iterrows():
    counter += 1
    if counter % ten_precent == 0:
        print(counter // ten_precent * 10, '% of the way done')
    row = row[1]
    _userId = row['userId']
    _movieId = row['movieId']
    user_frame.loc[_userId]['movie_vect'][_movieId] = row['rating']
'''

"\nusers_set = trimmed['userId'].unique()\nusers_set.sort()\nuser_frame = pd.DataFrame(users_set, columns = ['userId'])\nuser_frame = user_frame.set_index('userId')\n\nmovies_set = trimmed['movieId'].unique()\nmovies_set.sort()\nmovie_vector_blank = pd.DataFrame([np.zeros(len(movies_set))], columns = movies_set)\n\nuser_frame['movie_vect'] = [movie_vector_blank.copy() for i in range(user_frame.shape[0])]\n\ncounter = 0\nten_precent = int(trimmed.shape[0] / 10)\nfor row in trimmed.iterrows():\n    counter += 1\n    if counter % ten_precent == 0:\n        print(counter // ten_precent * 10, '% of the way done')\n    row = row[1]\n    _userId = row['userId']\n    _movieId = row['movieId']\n    user_frame.loc[_userId]['movie_vect'][_movieId] = row['rating']\n"

In [36]:
#old code
'''
user_num = 1
new_user = user_frame.loc[user_num]['movie_vect']
new_frame = user_frame.copy().drop(user_num)

sims = []
for row in new_frame.iterrows():
    userId = row[0]
    row = row[1]
    sims.append((userId, cosine_similarity(row['movie_vect'], new_user)[0][0]))
    
sims.sort(key = lambda x: x[1], reverse = True)
closest_user = int(sims[0][0])
closest_user = new_frame.iloc[closest_user]['movie_vect']

m = (closest_user * new_user).T
top_five = m[m[0] > 0].sort_values(by = 0, ascending = False)[0: 5]

for movie_id in top_five.index:
    top_five.loc[movie_id] = movies.set_index('movieId').loc[movie_id]['title']
'''
top_five

['Pulp Fiction (1994)',
 'Godfather, The (1972)',
 'Godfather: Part II, The (1974)',
 'Matrix, The (1999)',
 'Once (2006)']